<h1 style="font-size:2em; font-family:sans-serif">
  Laboratorio 5 - QKD
</h1>

<sub style="color: #888;">Adrian Rodriguez, 2025</sub>



Esta es una versión modificada del código usado para el ejercicio QKD, se explica mas a detalle en main.ipynb. 
Lo único nuevo es la construcción de la llave usando las bases que hacen match.

In [10]:
import random
import pandas as pd

ROUNDS = 15  # Number of rounds

def basis_symbol(b):  # 0: rectilinear (|), 1: diagonal (/)
    return '|' if b == 0 else '/'

def simulate_bb84(eve_interception_rate: float = 0.0):
    rows = []
    alice_key_bits = []
    bob_key_bits = []

    for _ in range(ROUNDS):
        # 1. Alice: chooses random bit and basis
        alice_bit = random.randint(0, 1)
        alice_basis = random.randint(0, 1)

        # 2. Eve intercepts?
        eve_intercepts = random.random() < eve_interception_rate
        if eve_intercepts:
            eve_basis = random.randint(0, 1)
            if eve_basis == alice_basis:
                eve_bit = alice_bit
            else:
                eve_bit = random.randint(0, 1)
            # Photon arrives at Bob with Eve's disturbance
            photon_bit = eve_bit
            photon_basis = eve_basis
        else:
            photon_bit = alice_bit
            photon_basis = alice_basis

        # 3. Bob chooses random basis and measures
        bob_basis = random.randint(0, 1)
        if bob_basis == photon_basis:
            bob_bit = photon_bit
        else:
            bob_bit = random.randint(0, 1)

        bases_match = (alice_basis == bob_basis)

        # Only keep bits for key where bases match
        if bases_match:
            alice_key_bits.append(alice_bit)
            bob_key_bits.append(bob_bit)

        rows.append({
            "Alice Bit": alice_bit,
            "Alice Basis": basis_symbol(alice_basis),
            "Bob Basis": basis_symbol(bob_basis),
            "Bob Bit": bob_bit,
            "Bases Match?": "Yes" if bases_match else "No",
            "Use for Key?": "Yes" if bases_match else "No"
        })

    df = pd.DataFrame(rows)

    # Calculate QBER (error rate in key bits)
    if alice_key_bits:
        errors = sum(a != b for a, b in zip(alice_key_bits, bob_key_bits))
        qber = 100 * errors / len(alice_key_bits)
    else:
        qber = 0.0

    print(df)
    print(f"\nAlice's key bits: {alice_key_bits}")
    print(f"Bob's key bits:   {bob_key_bits}")
    print(f"Final secret key (shared): ", end="")
    if alice_key_bits == bob_key_bits:
        print(alice_key_bits)
    else:
        # Show which bits are different
        print("".join([str(a) if a == b else "*" for a, b in zip(alice_key_bits, bob_key_bits)]))
        print("  *Note: '*' marks bits where Alice and Bob differ (error)")
    print(f"Number of bits in key: {len(alice_key_bits)}")
    print(f"Percentage of rounds kept: {100*len(alice_key_bits)/ROUNDS:.1f}%")
    print(f"QBER (error rate in key): {qber:.1f}%\n")

simulate_bb84(eve_interception_rate=0.2)  

    Alice Bit Alice Basis Bob Basis  Bob Bit Bases Match? Use for Key?
0           1           /         /        1          Yes          Yes
1           0           /         /        0          Yes          Yes
2           0           |         |        0          Yes          Yes
3           0           |         /        0           No           No
4           0           /         /        0          Yes          Yes
5           1           |         |        1          Yes          Yes
6           1           /         |        1           No           No
7           1           |         /        1           No           No
8           0           |         /        0           No           No
9           0           |         |        0          Yes          Yes
10          1           /         /        1          Yes          Yes
11          1           /         |        0           No           No
12          1           /         /        1          Yes          Yes
13    